# Data Description
The sensors were sampledat the highest rate each device would support, and we segment thisdata into non-overlapping windows of 128 time steps. We includethe data collected from the 31 smartphones in our experiments.

# Data Processing

In [2]:
import numpy as np
import torch 
import os
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from IPython.core.debugger import Tracer
scaler = StandardScaler()
debug_here = Tracer()

<ipython-input-2-ea82c6e7c2a6>:11: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`
  debug_here = Tracer()


In [3]:
data_dir = 'Activity recognition exp'
HHAR_phones = pd.read_csv(f'{data_dir}/Phones_accelerometer.csv')


In [33]:
def sliding_window(time_series, width, step, order='F'):
    w = np.hstack([time_series[i:1 + i - width or None:step] for i in range(0, width)])
    result = w.reshape((int(len(w) / width), width), order='F')
    if order == 'F':
        return result
    else:
        return np.ascontiguousarray(result)

def calc_normalization(data):
    num_instances, num_time_steps, num_features = data.shape
    data = np.reshape(data, (num_instances, -1))
    scaler.fit(data)
#     mean, std = (np.array([np.mean(x) for x in X_train], dtype=np.float32), np.array([np.std(x) for x in X_train], dtype=np.float32))
    return scaler
def apply_normalization(data, scaler):
#     scaler = StandardScaler()
    num_instances, num_time_steps, num_features = data.shape
    data = np.reshape(data, (num_instances, -1))
    norm_data = scaler.transform(data)
#     debug_here()
#     data = (data - mean) / (std + 1e-5)
    norm_data[np.isnan(norm_data)] = 0
    norm_data = np.reshape(norm_data, (num_instances, num_time_steps, num_features))
    return norm_data

def HHAR_data_generator(HHAR_phones, model_type):
    # dataloading 
#     HHAR_phones = pd.read_csv(f'{data_dir}/Phones_accelerometer.csv')
    full_data={}
    seq_length = 128
    shifting_step = 128
    num_variables = 3
    HHAR_phones = HHAR_phones.dropna()
#     HHAR_phones = pd.read_csv('Activity recognition exp/Phones_accelerometer.csv')
    # phones: convert categorical labels to numbers 
    HHAR_phones['gt'] = HHAR_phones['gt'].astype('category')
    HHAR_phones['labels']=HHAR_phones['gt'].cat.codes
    #HHAR_phones['labels'] += 1
    # drop irrelevant columns 
    HHAR_phones_refined = HHAR_phones.drop(columns=['Index', 'Arrival_Time', 'Creation_Time', 'Model','Device' , 'gt'])
    user_index = 0
    # looping trough users 
    for user_name, user_data in  HHAR_phones_refined.groupby('User'):
        data, labels = [], []
        # looping through data from each class per user
        for class_name, class_data in user_data.groupby('labels'):
            col_index=0
            # slicing data to 128 with no overlap 
            sliced_data = np.empty((int(class_data.shape[0]/seq_length),seq_length, num_variables))
            for column in class_data[['x', 'y', 'z']]:
                column_data = class_data[column]
                sliced_data[:,:,col_index] = sliding_window(column_data.values, seq_length,shifting_step, 'T')
                col_index +=1
           # append data 
            data.append(sliced_data)
            # generate labels
            class_labels = np.empty(sliced_data.shape[0])
            class_labels.fill(class_name)
            labels.append(class_labels.astype(int))
            
        # data and labels for each users 
        array_user_data= np.concatenate(data, axis=0 )
        array_user_labels= np.concatenate(labels, axis=0 )        
        
        # Stratified train, validation, test split of the data 
        X_train, X_test, y_train, y_test = train_test_split(array_user_data, array_user_labels,  stratify=array_user_labels,  test_size=0.3,random_state=1)
#         X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=1)
        
        # Data normalization 
        # Calculate mean and standard deviation based on train
        scaler = calc_normalization (X_train)
        
        # Apply normalization 
        X_train = apply_normalization(X_train,scaler)
#         X_val = apply_normalization(X_val,scaler)
        X_test = apply_normalization(X_test,scaler)
        
        # prepare samples
        train_data = {'samples':X_train, 'labels':y_train}
#         val_data   = {'samples':X_val, 'labels':y_val}
        test_data  = {'samples':X_test, 'labels':y_test}
        
        torch.save(train_data, f'../ADATIME_data/{model_type}/train_{user_index}.pt')
#         torch.save(val_data,  f'HHAR_user_data/val_{user_name}.pt')
        torch.save(test_data, f'../ADATIME_data/{model_type}/test_{user_index}.pt')
        user_index+=1

In [42]:
samsungold_data.shape[0]/128

17395.03125

In [31]:
data_dir = 'Activity recognition exp'
HHAR_phones = pd.read_csv(f'{data_dir}/Phones_accelerometer.csv')
nexus4_data=HHAR_phones[HHAR_phones['Model']=='nexus4']
s3_data=HHAR_phones[HHAR_phones['Model']=='s3']
s3mini_data=HHAR_phones[HHAR_phones['Model']=='s3mini']
samsungold_data=HHAR_phones[HHAR_phones['Model']=='samsungold']
devices =  {'nexus':nexus4_data, 's3':s3_data, 's3_mini':s3mini_data,'samsungold':samsungold_data}


In [63]:
HHAR_phones.shape[0]/128 *0.7

71435.41015625

In [64]:
HHAR_phones.shape[0]/128 *0.3

30615.17578125

In [57]:
samsungold_data.User.value_counts()  /128

g    2119.945312
b    2115.437500
e    2106.851562
f    2056.742188
i    2025.437500
c    1798.351562
h    1766.578125
a    1717.906250
d    1687.781250
Name: User, dtype: float64

In [51]:
samsungold_data.ushape[0]/128 *0.3

5218.509375

In [35]:
for data_name, dataset in devices.items():
    HHAR_data_generator(dataset,data_name)

<ipython-input-33-dcd3e5c7e928>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HHAR_phones['gt'] = HHAR_phones['gt'].astype('category')
<ipython-input-33-dcd3e5c7e928>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HHAR_phones['labels']=HHAR_phones['gt'].cat.codes
<ipython-input-33-dcd3e5c7e928>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [62]:
import math
import os
import random
import re
import sys


s = "aabbbccde"
out = list(s)
# [out(i) for i in range(len(out)) if i==j]
# [(i,j) for i,j in zip(out,out) if i==j]
x=[]
for i in range(len(out)):
    for j in range(len(out)):
        if i!=j
        
    x=[out[i] for j in range(len(out)) if out[i]==out[j]]
x 

['e']

In [78]:
import collections

d={k: 0 for k in list(s)}
for c in s:
    d[c] += 1
d={k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse = True)}   
for k,v in d.items():
    print (k,v)

b 3
a 2
c 2
d 1
e 1


In [81]:
OrderedCounter

__main__.OrderedCounter

In [4]:
from collections import Counter, OrderedDict

[print(*c) for c in  Counter(sorted(input())).most_common(3)]

aabbbccde
b 3
a 2
c 2


[None, None, None]

In [5]:
Counter()

collections.Counter

In [30]:
from functools import reduce
reduce(lambda x, y: x+y, ['mm', 'nn', 'ss', 'dd'])
# list(map(lambda x,y: x*2, [1,2,3,4,5]))
reduce(lambda x,y: 1*x+y, [1,2,3,4])

16.375

In [24]:
192/24


8.0